In [1]:
import re
import json
import ontospy
import networkx as nx
import matplotlib.pyplot as plt
from nltk.corpus import stopwords

stop_list = stopwords.words('english')

with open('data/thesaurus.json', 'r') as f:
    thesaurus = json.loads(f.read())
    thesaurus = thesaurus[0]
    
constitute = ontospy.Ontospy('data/ontology.owl')

In [2]:
# some functions to help manipulate the Constitute ontology

def build_constitute_edgelist(node, edgelist=[]):
    parent_lab = str(node.uri).split('/')[-1]
    children = node.children()
    
    for child in children:
        child_lab = str(child.uri).split('/')[-1]
        edgelist.append([parent_lab, child_lab])
        edgelist = build_constitute_edgelist(child, edgelist)
        
    
    return edgelist

def build_constitute_node_dic(node, node_dic={}):
    parent_lab = str(node.uri).split('/')[-1]
    if parent_lab not in node_dic:
        node_dic[parent_lab] = node.uri
        
    children = node.children()
    
    for child in children:
        node_dic = build_constitute_node_dic(child, node_dic)
        
    return node_dic

def build_constitute_label_dic(node, label_dic={}):
    node_label = constitute.getEntity(uri=node.uri).bestLabel().strip()
    node_label = ' '.join([w for w in node_label.lower().split() 
                           if w not in stop_list])
    if node_label not in label_dic:
        label_dic[node_label] = node.uri
        
    children = node.children()
    
    for child in children:
        label_dic = build_constitute_label_dic(child, label_dic)
        
    return label_dic

In [3]:
# some functions to manipulate the Venice thesaurus
def build_thesaurus_node_dic(node, node_dic={}):
    parent_lab = node.get('label')
    
    if parent_lab and parent_lab not in node_dic:
        parent_lab = re.sub('\<IT[-+]\>', '', parent_lab)
        parent_lab = ' '.join([w for w in parent_lab.lower().split() 
                               if w not in stop_list])
        node_dic[parent_lab] = {'id': node['id'], 'synonyms': node['synonyms']}
    
    for child in node['nodes']:
        node_dic = build_thesaurus_node_dic(child, node_dic)
        
    return node_dic

In [4]:
# class to try out various matching approaches

# base class
class MatchBase:
    def __init__(self, reference, word2vec_dir = None):
        import gensim
        self.ref_list = reference
        
        if word2vec_dir:
            self.word2vec_mod = gensim.models.Word2Vec.load(word2vec_dir)
            
        else:
            self.word2vec_mod = None
            
    def _preprocess(self, term):
        import string
        
        term = re.sub('[' + string.punctuation + ']', '', term)
        term = term.lower()
        
        return term

# matching on the node variable name
class NodeMatch(MatchBase):
    def str_match(self, word):
        # just look for simple equality between node IDs (after preprocessing)
        word = self._preprocess(word)
        preprocessed_ref = [self._preprocess(ref) for ref in self.ref_list]
        
        matches = [ref for ref in preprocessed_ref if ref == word]
        
        return matches
    
    def word2vec_match(self, word, cutoff=0.5):
        # use a pretrained word2vec model to calculate similarity
        # treat all values over cutoff as a match
        
        if not self.word2vec_mod:
            print('Be sure to load a word2vec model before using this function!')
        else:
            word = self._preprocess(word)
            preprocessed_ref = [self._preprocess(ref) for ref in self.ref_list]
            
            sims = {}
            
            for ref in preprocessed_ref:
                try:
                    sims[ref] = self.word2vec_mod.similarity(ref, word)
                except:
                    sims[ref] = None
            
            sims = {s: sims[s] for s in sims if sims[s] and sims[s] > cutoff}
            return sims
        
# matching on plain-text label
class LabelMatch(MatchBase):
    def word2vec_match(self, word, cutoff=0.9):
        # use a pretrained word2vec model to generate vectors for labels and terms
        # for multiword labels/terms, sum vectors for each term (not a great solution, but ok for testing)
        # treat all values over cutoff as a match
        
        from scipy.spatial.distance import cosine
    
        if not self.word2vec_mod:
            print('Be sure to load a word2vec model before using this function!')
        else:
            preprocessed_word = self._preprocess(word)
            summed_word_vec = sum([self.word2vec_mod[w] for w in preprocessed_word.split()
                                   if w in self.word2vec_mod])
            
            # sum() casts None to 0
            # so, checking for int is a hack to make sure at least one token in the word was in the word2vec vocab
            if type(summed_word_vec) != int:
                ref_vectors = {}
                for ref in self.ref_list:
                    preprocessed_ref = self._preprocess(ref)
                    summed_label = sum([self.word2vec_mod[w] for w in preprocessed_ref.split() 
                                        if w in self.word2vec_mod])
                    
                    # see above
                    if type(summed_label) != int:
                        ref_vectors[ref] = summed_label

                # note cosine = distance, not similarity w/ scipy
                sims = {ref: 1 - cosine(summed_word_vec, ref_vectors[ref]) for ref in ref_vectors}

                sims = {ref: sims[ref] for ref in sims if sims[ref] > cutoff}
            
                return sims
            
            else:
                return {}
    

In [5]:
# build the various helper dictionaries
constitute_base_node = constitute.getClass('/Topics')[0]
constitute_nodes = build_constitute_node_dic(constitute_base_node)
constitute_labels = build_constitute_label_dic(constitute_base_node)

# renaming node key in the top-level element to make recursion easier
try:
    thesaurus['nodes'] = thesaurus.pop('data')
except KeyError:
    pass

thesaurus_nodes = build_thesaurus_node_dic(thesaurus)

In [6]:
# run the two nodematch approaches
node_match = NodeMatch(constitute_nodes, 'data/word2vec_model_200_no_stem.pydata')

str_matches = []
word2vec_node_matches = {}
for node in thesaurus_nodes:
    str_matches += node_match.str_match(node)

    word2vec_match = node_match.word2vec_match(node)
    if word2vec_match:
        word2vec_node_matches[node] = word2vec_match

In [7]:
# Simple string match on node variable names works, but very limiting
str_matches

['flag',
 'immunity',
 'evidence',
 'religion',
 'motto',
 'budget',
 'elections',
 'finance',
 'quorum',
 'executive',
 'press',
 'legislation',
 'language',
 'opinion']

In [8]:
# word2vec on variable names works, but only works for labels that are in-vocab (e.g. "judcrts4" won't work)
# likely would perform better with a more broadly-trained model
# cutoff value optimization?

print('Number of matches:', len(word2vec_node_matches))
print('--------')
for node in word2vec_node_matches:
    print(node, word2vec_node_matches[node])
    print('\n')

Number of matches: 39
--------
admissibility {'jury': 0.50002017610264649}


subsidiarity {'artists': 0.58169680661190515, 'leisure': 0.5057359604314462}


opinion {'opinion': 1.0}


motto {'motto': 0.99999999999999989, 'flag': 0.55580798315594293, 'anthem': 0.56877491481239872}


proportionality {'artists': 0.54255512659428107}


press {'press': 0.99999999999999978}


foreigners {'slave': 0.54842999886666566, 'artists': 0.51293563607890402, 'asylum': 0.52807246732580326}


finance {'finance': 0.99999999999999978}


equality {'equal': 0.51332090408235009, 'dignity': 0.50960294834118935}


race {'religion': 0.64135900807386692}


correspondence {'privacy': 0.73245220338826944}


flag {'motto': 0.55580798315594293, 'anthem': 0.84562744610617557, 'arms': 0.54487966301844304, 'flag': 1.0}


executive {'executive': 1.0}


equity {'artists': 0.55882544590066585, 'leisure': 0.51139734863407682}


descent {'citizenship': 0.51500973580595866}


characteristics {'solid': 0.52293000243150789, 'sh

In [9]:
# run the label match approach
label_match = LabelMatch(constitute_labels, 'data/word2vec_model_200_no_stem.pydata')

word2vec_label_matches = {}
for node in thesaurus_nodes:
    word2vec_match = label_match.word2vec_match(node)
    if word2vec_match:
        word2vec_label_matches[node] = word2vec_match

In [10]:
# Not bad, though still not a ton of matches
# again, cutoff optimization?

print('Number of matches:', len(word2vec_label_matches))
print('--------')
for label in word2vec_label_matches:
    print(label, word2vec_label_matches[label])
    print('\n')

Number of matches: 65
--------
civil service {'civil service recruitment': 0.94226427866779272}


civil political rights {'civil political rights': 0.99999989369019582}


quasi-constitutional legislation {'legislation': 0.99999994125692659}


right environment {'right safe work environment': 0.91217489702483723}


central bank {'central bank': 1.000000023261278}


trade unions {'right join trade unions': 0.92007532169831341}


right self-determination {'right marry': 0.91258486410401041, 'right shelter': 0.90385679168888244}


right health {'right health care': 0.92349120316468525}


right petition {'right petition': 0.99999992146765215}


right resistance {'right marry': 0.94103134171541181, 'right shelter': 0.91216643457469215, 'right amparo': 0.90405590407667591}


supreme court {'supreme court': 1.0000000637617144, 'supreme court selection': 0.90037642165321397}


executive {'executive': 0.99999994113039825}


freedom association {'freedom association': 1.000000166888442}


interna

In [11]:
# Interestingly, pretty different information in two word2vec approaches
# combined, this gets ~10% of total
print('Number of overlaps:', sum([k in word2vec_label_matches for k in word2vec_node_matches]))

Number of overlaps: 7


In [12]:
print(len(constitute_nodes))
print(len(thesaurus_nodes))

390
708
